In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import missingno as msno

Carga y reducción de registros de datasets

In [ ]:
movies_df = pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\movies_df.csv")
director_actor_df = pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\Proyecto-individual-N-1---LABS---Henry\datos_procesados\director_actor_df.csv")

In [ ]:
# Eliminación de columnas que pueden haber sido útiles para los endpoints pero que no usaría como parámentro para recoemndar
movies_df = movies_df.drop(columns=['original_title', 'release_date', 'return'])

movies_df

## Análisis

Descripción

In [ ]:
movies_df.dtypes

In [ ]:
director_actor_df.dtypes

In [ ]:
# Primeras filas del dataset
print(movies_df.head())

# Información general del dataset
print(movies_df.info())

# Estadística descriptiva
print(movies_df.describe())


In [ ]:
director_actor_df = director_actor_df.drop(columns=['Unnamed: 0'])

print(director_actor_df.head())
print(director_actor_df.info())
print(director_actor_df.describe())

Chequeo valores nulos

In [ ]:
print(movies_df.isnull().sum())


In [ ]:
print(director_actor_df.isnull().sum())

Chequeo de duplicados

In [ ]:
duplicates = movies_df.duplicated()
print(duplicates)


In [ ]:
duplicates = director_actor_df.duplicated()
print(duplicates)

Chequeo de faltantes

In [ ]:
zero_as_missing = (director_actor_df == 0).sum()
print("Valores iguales a 0 contados como faltantes por columna:")
print(zero_as_missing)

In [ ]:
zero_as_missing = (movies_df == 0).sum()
print("Valores iguales a 0 contados como faltantes por columna:")
print(zero_as_missing)

<sus>Por esta razón, los análisis que tomen los valores de 'budget', 'revenue', 'vote average' y vote 'count' se tomarán con precaución.</sus>

## movies_df Gráficos

Histograma

In [ ]:
import matplotlib.pyplot as plt

# Recuento de la cantidad de películas por cada director
director_counts = director_actor_df['director'].value_counts()

# Límite a los primeros 20 directores más frecuentes para visualización
top_directors = director_counts.head(20)

# Creación del histograma
plt.bar(top_directors.index, top_directors.values, color='green', alpha=0.7)
plt.xlabel('Director')
plt.ylabel('Número de películas')
plt.title('Número de películas por director (Top 20)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


 <sub>A partir del director número 11, la cantidad de películas dirigidas comienza a disminuir más drásticamente. Por ejemplo, el director número 20 solo ha dirigido 50 películas, lo que es un 60% menos que la cantidad de películas dirigidas por John Ford, el director número 1.</sub>

Selección de columnas numéricas y cálculo de la matriz de correlación
<sub>(se consideran los resultados con precaución debido a los valores faltantes)</sub>

In [ ]:
# Elección de columnas para números
numeric_cols = ['budget', 'revenue', 'vote_average', 'vote_count']

# Conversión de valores no numericos a NaN
movies_df[numeric_cols] = movies_df[numeric_cols].replace('Not available', np.nan)

# Conversión a tipo numérico
movies_df[numeric_cols] = movies_df[numeric_cols].apply(pd.to_numeric)


# Cálculo de la matriz de correlación
correlation_matrix = movies_df[numeric_cols].corr()
print(correlation_matrix)

Heatmap

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de correlación entre variables numéricas')
plt.show()

<sub>Las películas con mayores ingresos tienden a tener un mayor número de votos, según revenue y vote_count.
Las películas con mayor presupuesto tienden a tener mayores ingresos, un mayor número de votos y una calificación promedio más alta.<sub/>



Gráfico de barras

In [ ]:
genre_counts = movies_df['genre'].value_counts()
genre_counts.plot(kind='bar', color='green', alpha=0.7)
plt.xlabel('Genres')
plt.ylabel('Número de películas')
plt.title('Número de películas por género')
plt.xticks(rotation=45)
plt.show()

<sub>Preferencia por ciertos géneros, con el drama y la comedia dominando significativamente.</sub>

Diagrama de dispersión

In [ ]:
plt.scatter(movies_df['budget'], movies_df['revenue'], alpha=0.5)
plt.xlabel('Budget')
plt.ylabel('Revenue')
plt.title('Budget vs Revenue')
plt.show()

<sub>Los mayores presupuestos suelen generar más ingresos. La dispersión de los puntos indica que tal relación no es siempre garantía de éxito. La mayoría de las películas se concentran en presupuestos e ingresos bajos, con algunos outliers entre la relación de presupuesto y retornos, pero no necesariamente son errores. Estos son algunos positivos y otros negativos entre la relación de valores.</sub>

Nube de palabras según el título de la película

In [ ]:
# Concatenación de todos los títulos en un solo string
text = ' '.join(movies_df['title'].astype(str))

# Creación de la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Muestra de la nube de palabras
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras por título')
plt.show()

Nube de palabras de género

In [ ]:
# Concatenación de todos los títulos en un solo string
text = ' '.join(movies_df['genre'].astype(str))

# Creación de la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Muestra de la nube de palabras
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras por género')
plt.show()

## director_actor_df Gráficos

Nube de palabras según directores

In [ ]:
director_names = director_actor_df['director'].str.split(' ')

# Concatenación de nombres y apellidos de directores en un solo string
text_directors = ' '.join([' '.join(name) for name in director_names])

# Creación de la nube de palabras para directores
wordcloud_directors = WordCloud(width=1000, height=400, background_color='white').generate(text_directors)

# Muestra de la nube de palabras para directores
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_directors, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras según directores')
plt.show()

Nube de palabras según actores

In [ ]:
actor_names = director_actor_df['actor'].str.split(' ')

# Concatenación de nombres y apellidos de actores en un solo string
text_actors = ' '.join([' '.join(name) for name in actor_names])

# Creación de la nube de palabras para actores
wordcloud_actors = WordCloud(width=800, height=400, background_color='white').generate(text_actors)

# Muestra de la nube de palabras para actores
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_actors, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de palabras según actores')
plt.show()